# Capital Asset Pricing Model on S&P 500

In [1]:
#import relevant packages
import pandas as pd
import numpy as np
import yfinance as yf
import math
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
yf.pdr_override()

In [2]:
#Importing S&P 500 Tickers
sp500 = pd.read_csv('s&p500_tickers.csv')
del sp500['Unnamed: 0']

In [3]:
print(sp500['Tickers'])

0       MMM
1       AOS
2       ABT
3      ABBV
4      ABMD
       ... 
498     YUM
499    ZBRA
500     ZBH
501    ZION
502     ZTS
Name: Tickers, Length: 503, dtype: object


In [4]:
#Datetime object for current date
today = datetime.today().strftime('%Y-%m-%d')
today 

'2022-09-01'

In [5]:
#Datetime object for one year ago
one_year = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
one_year

'2021-09-01'

# Base case before iteration

In [6]:
tickers = sp500['Tickers'][0]
start_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
test_price = round(yf.download(tickers, start=start_date, end=today)['Adj Close'][0],2)
test_price

[*********************100%***********************]  1 of 1 completed


124.86

In [7]:
sp500['Tickers'][0]

'MMM'

In [8]:
start_date = one_year 
end_date = today

tickers = sp500['Tickers'][0]
test_df = yf.download(tickers, start = start_date, end = end_date)['Adj Close']
test_df.head()

[*********************100%***********************]  1 of 1 completed


Date
2021-08-31    187.480499
2021-09-01    186.228943
2021-09-02    187.624893
2021-09-03    187.143555
2021-09-07    178.661957
Name: Adj Close, dtype: float64

In [9]:
start_date = one_year 
end_date = today

tickers = '^GSPC'
market_df = yf.download(tickers, start = start_date, end = end_date)['Adj Close']
market_df.head()

[*********************100%***********************]  1 of 1 completed


Date
2021-08-31    4522.680176
2021-09-01    4524.089844
2021-09-02    4536.950195
2021-09-03    4535.430176
2021-09-07    4520.029785
Name: Adj Close, dtype: float64

In [10]:
sec_df = pd.concat([test_df, market_df], axis=1)
sec_df.columns = [str(sp500['Tickers'][0]), 'Market']
sec_df

,MMM,Market
Date,,
2021-08-31,187.480499,4522.680176
2021-09-01,186.228943,4524.089844
2021-09-02,187.624893,4536.950195
2021-09-03,187.143555,4535.430176
2021-09-07,178.661957,4520.029785
...,...,...
2022-08-25,142.759995,4199.120117
2022-08-26,129.139999,4057.659912
2022-08-29,126.440002,4030.610107


In [11]:
#Expected Market Return
market_er = ((sec_df['Market'] / sec_df['Market'].shift(1)) - 1).mean() * 252
market_er

-0.11202895868380602

In [12]:
#Risk free rate as the 10-Year US Treasury Bill
start_date = one_year 
end_date = today

tickers = '^TNX'
rf_rate = round(yf.download(tickers, period='all')['Adj Close'][0],2)
rf_rate

[*********************100%***********************]  1 of 1 completed


3.27

In [13]:
#Beta Calculation
sec_returns = np.log(sec_df / sec_df.shift(1) )
cov = sec_returns.cov() * 250
cov_with_market = cov.iloc[0,1]
market_var = sec_returns['Market'].var() * 250

test_beta = cov_with_market / market_var
test_beta

0.6178821041622915

In [14]:
#CAPM Expected Return Calculation
test_er = rf_rate + test_beta * (market_er-rf_rate)
test_er

1.180304830670646

In [15]:
my_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Beta',
                'CAPM Expected Return',
                
                ]
test = pd.DataFrame(columns = my_columns)
test

,Ticker,Price,Number of Shares to Buy,Beta,CAPM Expected Return


In [16]:
test= test.append(
                                            pd.Series([str(sp500['Tickers'][0]), 
                                                        test_price, 
                                                        'N/A',
                                                        test_beta,
                                                        test_er, 
                                                       ], 
                                                      index = my_columns), 
                                            ignore_index = True)

In [17]:
test

,Ticker,Price,Number of Shares to Buy,Beta,CAPM Expected Return
0,MMM,124.86,N/A,0.617882,1.180305


#  Iterating through S&P 500 tickers

In [18]:
my_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Beta',
                'CAPM Expected Return'
                ]
capm_df = pd.DataFrame(columns = my_columns)
capm_df

,Ticker,Price,Number of Shares to Buy,Beta,CAPM Expected Return


In [19]:
for t in sp500['Tickers']:
    try:
        tickers = t
        start_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
        price = round(yf.download(tickers, start=start_date, end=today)['Adj Close'][0],2)
        
        ticker_df = yf.download(tickers, start = one_year, end = today)['Adj Close']
        market_df = yf.download('^GSPC', start=one_year, end=today)['Adj Close']
        
        sec_df = pd.concat([ticker_df, market_df], axis=1)
        sec_df.columns = [str(t), 'Market']
        
        rf_rate = round(yf.download('^TNX', period='all')['Adj Close'][0],2)
        rf_rate
        
        sec_returns = np.log(sec_df / sec_df.shift(1) )
        cov = sec_returns.cov() * 250
        cov_with_market = cov.iloc[0,1]
        market_var = sec_returns['Market'].var() * 250
        beta = cov_with_market / market_var
        
        capm_er = rf_rate + beta * (market_er - rf_rate)


        capm_df = capm_df.append(
                                            pd.Series([t, 
                                                        price, 
                                                        'N/A', 
                                                        beta,
                                                        capm_er
                                                       ], 
                                                      index = my_columns), 
                                            ignore_index = True)
    except IndexError:
        continue

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

...

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
capm_df

,Ticker,Price,Number of Shares to Buy,Beta,CAPM Expected Return
0,MMM,124.86,N/A,0.617882,1.180306
1,AOS,57.44,N/A,0.944680,0.075064
2,ABT,102.20,N/A,0.783687,0.619549
3,ABBV,135.55,N/A,0.316451,2.199755
4,ABMD,258.65,N/A,1.566260,-2.027136
...,...,...,...,...,...
496,YUM,110.78,N/A,0.745898,0.744810
497,ZBRA,302.86,N/A,1.473793,-1.709672
498,ZBH,107.37,N/A,0.823089,0.484521
499,ZION,55.40,N/A,1.070746,-0.350586


# Selecting the 50 Best CAPM Stocks

In [21]:
top_50_capm_df = capm_df.sort_values(by = ['CAPM Expected Return'], ascending = False).copy()[:51]
top_50_capm_df = top_50_capm_df.reset_index(drop=True)

# Calculating the Number of Shares to Buy

In [22]:
#Assuming portfolio size of $10,000,000
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:10000000
10000000


In [23]:
position_size = float(portfolio_size) / len(top_50_capm_df)
for i in range(len(top_50_capm_df)):
    top_50_capm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / top_50_capm_df['Price'][i])
top_50_capm_df

,Ticker,Price,Number of Shares to Buy,Beta,CAPM Expected Return
0,K,72.61,2700,0.065535,3.048358
1,NEM,42.20,4646,0.097675,2.939662
2,HRL,50.66,3870,0.157942,2.735834
3,KMB,128.83,1521,0.193687,2.614946
4,CPB,50.69,3868,0.219954,2.526109
5,LMT,422.31,464,0.219747,2.519006
6,SJM,140.81,1392,0.231017,2.488695
7,KHC,37.41,5241,0.236838,2.469007
8,ATVI,78.62,2494,0.239558,2.459809
9,WEC,103.47,1895,0.242739,2.441478


# Formatting Our Excel Output

In [24]:
writer = pd.ExcelWriter('capm_stocks.xlsx', engine='xlsxwriter')
top_50_capm_df.to_excel(writer, sheet_name='CAPM Strategy', index = False)

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0.0000',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.00%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Beta', integer_template],
                    'E': ['CAPM Expected Return', integer_template],
                    }

for column in column_formats.keys():
    writer.sheets['CAPM Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['CAPM Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [27]:
writer.save()